In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import os
from nltk.stem import WordNetLemmatizer
from nltk.metrics.distance import edit_distance as levenshteinDistance

C:\Users\JS\AppData\Local\Temp\ipykernel_42040\3151377594.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [40]:
VAR = {
    'data_path': os.path.join('UpdatedResumeDataSet_T1_7.csv'),
}

In [4]:
res_data = pd.read_csv(VAR['data_path'])

In [5]:
res_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9595 entries, 0 to 9594
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  9595 non-null   object
 1   Resume    9595 non-null   object
dtypes: object(2)
memory usage: 150.1+ KB


In [6]:
res_data.head(5)

,Category,Resume
0,Data Science,qwtnrvduof Education Details \nMay 2013 to May...
1,Data Science,"qwtnrvduof Areas of Interest Deep Learning, Co..."
2,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
3,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."
4,Data Science,"SKILLS C Basics, IOT, Python, MATLAB, Data Sci..."


### Explore Resume Text

In [62]:
sample_res = res_data['Resume'][0]
print(sample_res)

qwtnrvduof Education Details 
May 2013 to May 2017 BbNTGBqLmkKE   UIT-RGPV
Data Scientist 

Data Scientist - Matelabs
Skill Details 
Python- Exprience - Less than 1 year months
Statsmodels- Exprience - 12 months
AWS- Exprience - Less than 1 year months
Machine learning- Exprience - Less than 1 year months
Sklearn- Exprience - Less than 1 year months
Scipy- Exprience - Less than 1 year months
Keras- Exprience - Less than 1 year monthsCompany Details 
company - Matelabs
description - ML Platform for business professionals, dummies and enthusiastsckeKJOFvWQ
60/A Koramangala 5th block,
Achievements/Tasks behind sukh sagar, Bengaluru,
India                               Developed and deployed auto preprocessing steps of machine learning mainly missing value
treatment, outlier detection, encoding, scaling, feature selection and dimensionality reductionqunsOBcUdT
Deployed automated classification and regression modelRYNOolXhuV
linkedinSAJhwmUxoOcom/in/aditya-rathore-
b4600b146                

Initial Review:
Some words are seemingly gibberish and consists of a sequence of random characters

These words should be removed. However, care must be taken to ensure that other important text such as links are not classified as gibberish

List of issues:
Broken links (Solved)
Long whitespaces (Solved)
Combined words without clear separators https://github.com/grantjenks/python-wordsegment

In [63]:
sample_res

'qwtnrvduof Education Details \nMay 2013 to May 2017 BbNTGBqLmkKE   UIT-RGPV\nData Scientist \n\nData Scientist - Matelabs\nSkill Details \nPython- Exprience - Less than 1 year months\nStatsmodels- Exprience - 12 months\nAWS- Exprience - Less than 1 year months\nMachine learning- Exprience - Less than 1 year months\nSklearn- Exprience - Less than 1 year months\nScipy- Exprience - Less than 1 year months\nKeras- Exprience - Less than 1 year monthsCompany Details \ncompany - Matelabs\ndescription - ML Platform for business professionals, dummies and enthusiastsckeKJOFvWQ\n60/A Koramangala 5th block,\nAchievements/Tasks behind sukh sagar, Bengaluru,\nIndia                               Developed and deployed auto preprocessing steps of machine learning mainly missing value\ntreatment, outlier detection, encoding, scaling, feature selection and dimensionality reductionqunsOBcUdT\nDeployed automated classification and regression modelRYNOolXhuV\nlinkedinSAJhwmUxoOcom/in/aditya-rathore-\nb46

In [64]:
def clean_links(potentialLinks: list):
    
    '''
    Assumption: Potential link will always have at the minimum a .com
    
    Checks validity of link and returns cleaned link string
    '''
    
    assert isinstance(potentialLinks, list)
    
    http_exist = False
    www_exist = False
    com_exist = False
    
    if len(potentialLinks) < 1:
        return []
    
    ret_list = []
    
    for link in potentialLinks:
        
        http_match = re.search(r'(https?)(:)?(\/){0,2}', link)
        www_match = re.search(r'(www)(\.)?', link)
        com_match = re.search(r'(\.)?(com)', link)
        
        #http
        if http_match != None:
            http_exist = True
        
        #www
        if www_match != None:
            www_exist = True
        
        #com
        if com_match != None:
            com_exist = True
            
        if (com_exist) or (com_exist and www_exist) or (com_exist and www_exist and http_exist):
            link = re.sub(r'(https?)(:)?(\/){0,2}', 'https://', link)
            link = re.sub(r'(www)(\.)?', 'www.', link)
            link = re.sub(r'(\.)?(com)', '.com', link)
            
            ret_list.append(link)
        else:
            #Not valid link
            ret_list.append(False)
            
    return ret_list

In [65]:
def clean_raw_text(text: str):
    
    # Clean links section
    potential_links = re.findall(
        r'(?:(?:https?:?\/\/{1,2})?w{1,3}\.?)?[a-zA-z0-9]{1,2048}\.?[a-zA-Z0-9]{1,6}\b[/\-a-zA-Z0-9]*\w', text
    ) 
    '''
    / will flag a sequence of characters as potential links
    
    Optional criteions: 
    http(s)
    //
    www & .
    . & com
    '''
    
    finalized_links = clean_links(potential_links)

    for potential_link, finalized_link in zip(potential_links, finalized_links):
        if finalized_link == False:
            continue
        else:
            text = re.sub(potential_link, finalized_link, text)
    
    #Clean non-characters
    text = re.sub(r'[^a-zA-Z0-9]', r' ', text)
    
    #Normalize text
    text = text.lower()

    #Clean whitespace section
    text = re.sub(r'[ ]{1,}', r' ', text)
    
    return text

clean_raw_text(sample_res)

'qwtnrvduof education details may 2013 to may 2017 bbntgbqlmkke uit rgpv data scientist data scientist matelabs skill details python exprience less than 1 year months statsmodels exprience 12 months aws exprience less than 1 year months machine learning exprience less than 1 year months sklearn exprience less than 1 year months scipy exprience less than 1 year months keras exprience less than 1 year monthscompany details company matelabs description ml platform for business professionals dummies and enthusiastsckekjofvwq 60 a koramangala 5th block achievements tasks behind sukh sagar bengaluru india developed and deployed auto preprocessing steps of machine learning mainly missing value treatment outlier detection encoding scaling feature selection and dimensionality reductionqunsobcudt deployed automated classification and regression modelrynoolxhuv linkedinsajhwmuxoo com in aditya rathore b4600b146 reasearch and deployed the time series forecasting model arima sarimax holt winter and

In [66]:
def clean_structured_text(text: list | np.ndarray):
    
    #TODO There may be no point to cleaning mistyped random words > Intefere with keywords > Model may have to simply learn the noise

    if isinstance(text, list):
        text = np.array(text)
    elif isinstance(text, np.ndarray):
        pass
    else:
        raise TypeError('Not a list or numpy array')
    
    english_dictionary = nltk.corpus.words.raw().split('\n')
    english_dictionary = [word.lower() for word in english_dictionary] # normalize to lowercase
    
    word_in_dict_bool = np.isin(text, english_dictionary)
    
    word_not_in_dict_bool = np.invert(word_in_dict_bool)
    
    
    
    words_in_dict = text[word_not_in_dict_bool]
    
    print(words_in_dict)

clean_structured_text(sample_lemmas)

['iabac' '1' 'solutionsbntgbqlmkk' 'fahed' 'robotics' 'etcckekjofvwq'
 'solutionsqunsobcudteducation' '2017' 'brynoolxhuv' 'mohali' 'punjab'
 'indo' 'datamites' 'exprience' '13' 'exprience' '24' 'exprience' '24'
 'exprience' '24' 'exprience' '24' 'exprience' '24' 'monthscompany'
 'datamites' 'toolssajhwmuxoo' 'extracting' 'dataiqmadshiyn' 'processing'
 'pythoniszogerzlf' 'visualizationbrbegnceae' 'pvt' 'ltd' 'processing'
 'analyzed']


In [67]:
def wordnet_tag_format(tag: str):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('V'):
        return 'v'
    if tag.startswith('A'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    
    return 'n' #Ensure lemmatize function can run

In [68]:
def extract_lemmas(tagged_tokens: list[tuple], lemmatizer):
    lemmas = [lemmatizer.lemmatize(token[0], wordnet_tag_format(token[1])) for token in tagged_tokens]
    
    return lemmas

In [69]:
sample_tokens = nltk.tokenize.word_tokenize(clean_raw_text(sample_res))
sample_tokens[:10]

['qwtnrvduof',
 'education',
 'details',
 'may',
 '2013',
 'to',
 'may',
 '2017',
 'bbntgbqlmkke',
 'uit']

In [70]:
sample_tag = nltk.tag.pos_tag(sample_tokens)
sample_tag[:10]

#https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

[('qwtnrvduof', 'JJ'),
 ('education', 'NN'),
 ('details', 'NNS'),
 ('may', 'MD'),
 ('2013', 'CD'),
 ('to', 'TO'),
 ('may', 'MD'),
 ('2017', 'VB'),
 ('bbntgbqlmkke', 'NN'),
 ('uit', 'NN')]

In [71]:
sample_ner = nltk.ne_chunk(sample_tag)

sample_ner_categories = set()

for chunk in sample_ner:
    if isinstance(chunk, nltk.tree.Tree):
        sample_label = chunk.label()
        sample_token = ''.join(word for word, tag in chunk.leaves())
        print(f'word_token: {sample_token}; category: {sample_label}')
        sample_ner_categories.add(sample_label)

# OUT OF CODE

In [58]:
lemmatizer = WordNetLemmatizer()

In [59]:
sample_lemmas = extract_lemmas(sample_tag, lemmatizer)
sample_lemmas

['skill',
 'python',
 'tableau',
 'data',
 'visualization',
 'r',
 'studio',
 'machine',
 'learn',
 'statistic',
 'iabac',
 'certify',
 'data',
 'scientist',
 'with',
 'versatile',
 'experience',
 'over',
 '1',
 'year',
 'in',
 'manage',
 'business',
 'data',
 'science',
 'consulting',
 'and',
 'lead',
 'innovation',
 'project',
 'bring',
 'business',
 'idea',
 'to',
 'work',
 'real',
 'world',
 'solutionsbntgbqlmkk',
 'be',
 'a',
 'strong',
 'advocator',
 'of',
 'augment',
 'era',
 'where',
 'human',
 'capability',
 'be',
 'enhance',
 'by',
 'machine',
 'fahed',
 'be',
 'passionate',
 'about',
 'bring',
 'business',
 'concept',
 'in',
 'area',
 'of',
 'machine',
 'learn',
 'ai',
 'robotics',
 'etcckekjofvwq',
 'to',
 'real',
 'life',
 'solutionsqunsobcudteducation',
 'detail',
 'january',
 '2017',
 'brynoolxhuv',
 'tech',
 'computer',
 'science',
 'engineering',
 'mohali',
 'punjab',
 'indo',
 'global',
 'college',
 'of',
 'engineering',
 'data',
 'science',
 'consultant',
 'data',
 '

In [60]:
sample_lemmas

['skill',
 'python',
 'tableau',
 'data',
 'visualization',
 'r',
 'studio',
 'machine',
 'learn',
 'statistic',
 'iabac',
 'certify',
 'data',
 'scientist',
 'with',
 'versatile',
 'experience',
 'over',
 '1',
 'year',
 'in',
 'manage',
 'business',
 'data',
 'science',
 'consulting',
 'and',
 'lead',
 'innovation',
 'project',
 'bring',
 'business',
 'idea',
 'to',
 'work',
 'real',
 'world',
 'solutionsbntgbqlmkk',
 'be',
 'a',
 'strong',
 'advocator',
 'of',
 'augment',
 'era',
 'where',
 'human',
 'capability',
 'be',
 'enhance',
 'by',
 'machine',
 'fahed',
 'be',
 'passionate',
 'about',
 'bring',
 'business',
 'concept',
 'in',
 'area',
 'of',
 'machine',
 'learn',
 'ai',
 'robotics',
 'etcckekjofvwq',
 'to',
 'real',
 'life',
 'solutionsqunsobcudteducation',
 'detail',
 'january',
 '2017',
 'brynoolxhuv',
 'tech',
 'computer',
 'science',
 'engineering',
 'mohali',
 'punjab',
 'indo',
 'global',
 'college',
 'of',
 'engineering',
 'data',
 'science',
 'consultant',
 'data',
 '

In [19]:
lemmatizer.lemmatize('deployed', 'v')

'deploy'

In [20]:
sample_tag

[('qwtnrvduof', 'JJ'),
 ('education', 'NN'),
 ('details', 'NNS'),
 ('may', 'MD'),
 ('2013', 'CD'),
 ('to', 'TO'),
 ('may', 'MD'),
 ('2017', 'VB'),
 ('bbntgbqlmkke', 'NN'),
 ('uit', 'NN'),
 ('rgpv', 'NN'),
 ('data', 'NNS'),
 ('scientist', 'NN'),
 ('data', 'NNS'),
 ('scientist', 'NN'),
 ('matelabs', 'NN'),
 ('skill', 'NN'),
 ('details', 'NNS'),
 ('python', 'VBP'),
 ('exprience', 'NN'),
 ('less', 'JJR'),
 ('than', 'IN'),
 ('1', 'CD'),
 ('year', 'NN'),
 ('months', 'NNS'),
 ('statsmodels', 'NNS'),
 ('exprience', 'RB'),
 ('12', 'CD'),
 ('months', 'NNS'),
 ('aws', 'JJ'),
 ('exprience', 'NN'),
 ('less', 'JJR'),
 ('than', 'IN'),
 ('1', 'CD'),
 ('year', 'NN'),
 ('months', 'NNS'),
 ('machine', 'NN'),
 ('learning', 'VBG'),
 ('exprience', 'NN'),
 ('less', 'JJR'),
 ('than', 'IN'),
 ('1', 'CD'),
 ('year', 'NN'),
 ('months', 'NNS'),
 ('sklearn', 'VBP'),
 ('exprience', 'NN'),
 ('less', 'JJR'),
 ('than', 'IN'),
 ('1', 'CD'),
 ('year', 'NN'),
 ('months', 'NNS'),
 ('scipy', 'VBP'),
 ('exprience', 'NN'),
 

In [21]:
# Watch Cell
# print(clean_raw_text(sample_res))
# clean_raw_text(sample_res)
# nltk.corpus.words.raw().split('\n')
# np.isin(['detail'], nltk.corpus.words.raw().split('\n'))]
def x():
    x_dict = nltk.corpus.words.raw().split('\n')
    x_list = [levenshteinDistance('sklearn', word) for word in x_dict]
    
    id = x_list.index(min(x_list))
    print(id)
    print(min(x_list))
    print(nltk.corpus.words.raw().split('\n')[id])
    
x()

# lemmatizer.lemmatize('extracting')

105091
2
learn


In [22]:
from nltk.metrics.distance import edit_distance as test
test('aws', 'reductionqunsobcudt', substitution_cost=1)

18